In [1]:
import json
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from datasets import Dataset

2024-04-07 16:30:22.929090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = "./data/additional_datas.json"

with open(DATA_PATH, "r") as fichier:
    data = json.load(fichier)


df = pd.DataFrame.from_dict(data, orient='index')
df = df.stack().reset_index(level=1, drop=True).reset_index()
df.columns = ['Label', 'Text']
df = df[['Text', 'Label']]

numerize_labels = {'Politics':0, 'Health':1, 'Finance':2, 'Travel':3, 'Food':4, 'Education':5,
       'Environment':6, 'Fashion':7, 'Science':8, 'Sports':9, 'Technology':10, 'Entertainment':11}

df.Label = df.Label.map(numerize_labels)
df.head()

,Text,Label
0,The mayor announced a new initiative to improv...,0
1,The senator is facing criticism for her stance...,0
2,The upcoming election has sparked intense deba...,0
3,The governor unveiled a new plan to boost loca...,0
4,The congressman is under scrutiny for his comm...,0


In [3]:

# Step 1: Split the dataset into training and validation sets
train_dataset = pd.DataFrame(columns=['Text', 'Label'])
val_dataset = pd.DataFrame(columns=['Text', 'Label'])

for key in numerize_labels:
    df_key = df[df['Label'] == numerize_labels[key]]
    train_dataset = pd.concat([train_dataset, df_key.sample(frac=0.8)])
    val_dataset = pd.concat([val_dataset, df_key.drop(df_key.sample(frac=0.8).index)])
        
# Step 2: Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(numerize_labels))
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device : ", device)

# Move the model to the device
model.to(device)

# Step 3: Tokenize the text data using the BERT tokenizer
train_encodings = tokenizer(list(train_dataset['Text']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_dataset['Text']), truncation=True, padding=True)

# Step 4: Convert the tokenized data into input features compatible with BERT
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': list(train_dataset['Label'])})
val_dataset = Dataset.from_dict({'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask'], 'labels': list(val_dataset['Label'])})

# Step 5: Define the model architecture
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=200,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Step 6: Train the model
trainer.train()

# Step 7: Evaluate the model
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device :  cpu


/Users/etienne/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/28200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("./model")

In [ ]:
result = pd.DataFrame(columns=['ID', 'Label'])

file_path = "./data/test_shuffle.txt"

with open(file_path, "r") as file:
    text = file.read()

for i, sentence in enumerate(text.split("\n")):
    inputs = tokenizer.encode_plus(sentence, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits).item()
    predicted_label_name = list(numerize_labels.keys())[predicted_label]
    result = pd.concat([result, pd.DataFrame({'ID': [i], 'Label': [predicted_label_name]})], ignore_index=True)

result = result.iloc[:-1]
result.to_csv("result3.csv", index=False)